# Recurrent Neural Networks

Recurrent neural network is a type of network architecture that accepts variable inputs and variable outputs, which contrasts with the vanilla feed-forward neural networks. 

We can also consider input with variable length, such as video frames and we want to make a decision along every frame of that video.

## Process Sequences
![sequence](img/sequence.png)

#### one to one
This is the classic feed forward neural network architecture, with one input and we expect one output.

#### one to many
This can be thought of as image captioning. We have one image as a fixed size input and the output can be words or sentences which are variable in length.

#### many to one
This is used for sentiment classification. The input is expected to be a sequence of words or even paragraphs of words. The output can be a regression output with continuous values which represent the likelihood of having a positive sentiment.

#### many to many
This model is ideal for machine translation like the one we see on Google translate. The input could an English sentence which has variable length and the output will be the same sentence in a different language which also has variable length. 

The last many to many model can be used for video classification on frame level. Feed every frame of a video into the neural network and expect an output right away. However, since frames are generally dependent on each other, it is necessary for the network to propagate its hidden state from the previous to the next. Thus, we need recurrent neural network for this kind of task.

## Mathematical Formulation
We can process a sequence of vectors **x** applying a recurrence formula at every time step:

$$
h_{t} = f_{W}(h_{t - 1}, x_{t})
$$

`x[t]` is input vector at some time step while `h[t - 1]` is the previous hidden state and `h[t]` is the new hidden state produced by the network. The whole forward propagation can be characterized by the function of **W**.

**NOTE**: The same function and same set of parameters are used at every time step.

### Example
Here's a simple vanilla recurrent neural network example in functional form. Notice that if we were to produce `h[t]`, we need some weight matrices, `h[t-1]`, `x[t]` and a non-linearity `tanh`.

$$
h_{t} = tanh(W_{hh}h_{t-1} + W_{xh}x_{t})
$$

If we want to produce an output at every timestep, then we want another weight matrix that accepts a hidden state and project it to an output `y`

$$
y_{t} = W_{hy}h_{t}
$$

In [17]:
import numpy as np


np.random.seed(0)
class RecurrentNet(object):
    """When we say W_hh, it means a weight matrix that accepts a hidden state and produce a new hidden state. 
    Similarly, W_xh represents a weight matrix that accepts an input vector and produce a new hidden state.
    """
    def __init__(self):
        self.hidden_state = np.zeros((3, 3))
        self.W_hh = np.random.randn(3, 3)
        self.W_xh = np.random.randn(3, 3)
        self.W_hy = np.random.randn(3, 3)
    
    def forward_prop(self, x):
        self.hidden_state = np.tanh(self.W_hh.dot(self.hidden_state) + self.W_xh.dot(x))
        
        return self.W_hy.dot(self.hidden_state)

In [16]:
input_vector = np.ones((3, 3))
silly_network = RecurrentNet()

# Notice that same input, but leads to different ouptut at every single time step.
print silly_network.forward_prop(input_vector)
print silly_network.forward_prop(input_vector)
print silly_network.forward_prop(input_vector)

[[-2.80121891 -2.80121891 -2.80121891]
 [ 0.69469708  0.69469708  0.69469708]
 [ 0.96886448  0.96886448  0.96886448]]
[[-3.04406657 -3.04406657 -3.04406657]
 [ 0.7898616   0.7898616   0.7898616 ]
 [ 0.86068273  0.86068273  0.86068273]]
[[-3.04498651 -3.04498651 -3.04498651]
 [ 0.78981591  0.78981591  0.78981591]
 [ 0.86049092  0.86049092  0.86049092]]


## Computational Graph
Instead of imagining that hidden state is being *recurrently* fed back into the network, it's easier to visualize the process if we unroll the operation into a computational graph that is composed to many time steps.

For example, we begin with a zero'ed vector as our hidden state on the left. We feed it into the network along with our first input. When we receive the next input, we take the new hidden state and feed it into the network again with the second input. The procoess goes on until the point we wish to compute the final output of the network.

![computational-graph-1](img/computational-graph-1.png)

We use the same set of weight for every time step of the computation.

![computational-graph-2](img/computational-graph-2.png)

### Computational Graph - Many to Many
For the many-to-many case, we compute a `y[t]` and the loss for every time step. At the end we simply sum up the loss of all the time steps and count that as our total loss of the network. 

When we think about the back propagation for this model, we will have a separate gradient for W flowing from each of those time steps and then the final gradient for W will be the sum of all those individual time step gradients. *Imagine that we have some sort of ground-truth label for every step of the sequence*:


![computational-graph-many-to-many](img/computational-graph-many-to-many.png)



### Computational Graph - Many to One
If we have this many to one situation, we make the decision based on the final hidden state of this network. This final hidden state summarizes all of the context from the entire sequence. 


![computational-graph-many-to-one](img/computational-graph-many-to-one.png)

### Computational Graph - One to Many
If we have this one to many situation, where we want to receive a fixed size input and produce a variable length output, then you would commonly use that fixed size input to initialize the hidden state and then let the network to propagate and evolve the hidden state forward. 


![computational-graph-one-to-many](img/computational-graph-one-to-many.png)

## Squence to Sequence
For the sequence to sequence models where you might want to do something like machine translation, this is a combination of **many-to-one** and **one-to-many** architecture. We proceed in two stages, (1) the encoder receives a variably sized input like an english sentence and performs encoding into a hidden state vector, (2) the decoder receives the hidden state vector and produces a variably sized output. The motivation of using this architecture is modularity. We can easily swap out encoder and decoder for different type of language translation.

## Character-leve Languag Model
### Training Time
Suppose that we have a character-level language model, the list of possible *vocabularies* is `['h', 'e', 'l', 'o']`.  An example training sequence is `hello`. The same output from hidden layer is being fed to output layer and the next hidden layer, as noted below that `y[t]` is a product of `W_hy` and `h[t]`. Since we know what we are expecting, we can backpropagate the cost and update weights.

The `y[t]` is a prediction for which letter is most likely to come next. For example, when we feed `h` into the network, `e` is the expected output of the network because the only training example we have is `hello`. 

![language-model](img/language-model.png)

### Test Time
At test time, we sample characters one at a time and feed it back to the model to produce a whole sequence of characters (which makes up a word.) We seed the word with a prefix like the letter **h** in this case. The output is a softmax vector which represents probability. We can use it as a probability distribution and perform sampling.

**This means EACH character has some chance to be selected** Samplng technique gives us more diversity in the output. This is evident in sentence construction. Given a prefix, we can have multiple words and phrases to represent the same idea.

![language-model-test-time](img/language-model-test-time.png)

## Backpropagation
The concept of backpropagation through time is that we forward through entire sequence to compute loss, then backward through entire sequence to compute gradient. However, this becomes problematic when we want to train a sequence that is very long. For example, if we were to train a a paragraph of words, we have to iterate through many layers before we can compute one simple gradient step.

### Truncated Backprop
In practice, what people do is an approximation called truncated backpropagation through time. Run forward and backward through chunks of the sequence instead of the whole sequence. Even though our input sequence can potentially be very long or even infinite, when we are training our model, we will step forward for some number of steps and compute a loss only over this sub sequence of the data. Then backpropagate through this sub-sequence and make a gradient step on the weights. 

When we move to the next batch, we still have this hidden state from the previous batch of data, we will carry this hidden state forward. The forward pass is unaffected but we will only backpropgate again through this second batch. 

![